In [1]:
from pxr import Usd, UsdGeom, UsdUtils, UsdShade, Sdf, Gf
import struct
from pathlib import Path
from script import extract_gltf_from_b3dm, \
                   check_converter_availability, \
                   create_layer_from_geotiff, \
                   create_layer_from_3dtiles, \
                   create_stage, \
                   add_layer_to_stage, \
                   save_stage, \
                   create_layer_from_geopackage
import rasterio
import getpass

In [2]:
# This is needed to see something in usdview as values in lat/lon coordinates
# show up clustered and are almost invisible.  Omniverse shold allow using no SCALE_FACTOR ?
SCALE_FACTOR = 100000.0

# Tests

In [ ]:
stage = Usd.Stage.CreateNew('HelloWorld.usda')
xformPrim = UsdGeom.Xform.Define(stage, '/hello')
spherePrim = UsdGeom.Sphere.Define(stage, '/hello/world')
stage.GetRootLayer().Save()

# Extract glTF from 3DTiles/b3dm

In [3]:
full_path = getpass.getpass("Enter path to b3dm file: ")
b3dm_file = Path(full_path) # Autobuilding lidar

In [ ]:
print(f"Extracting glTF data from {b3dm_file} to its corresponding .glb file")
extract_gltf_from_b3dm(b3dm_file)

In [5]:
gltf_file = b3dm_file.with_suffix('.glb')

In [6]:
# Create a new USD stage from the GLB file
gltf_path = str(gltf_file)  # Convert Path to string
output_usd = gltf_file.with_suffix('.usda')

# Convert a small DEM to USD

In [3]:
dem_file = Path("02EJ_5m_Cilp_4326.tif")

In [4]:
usd_output_file = dem_file.with_suffix('.usdc')
# usd_filename = "usd_output_file.usda"
# usd_output_file = Path(usd_filename)

In [5]:
# 1. Create a new stage
stage = create_stage(usd_output_file)

In [6]:
# 2. Create the DEM layer from GeoTIFF
dem_layer = create_layer_from_geotiff(dem_file, 
           sample_factor=1, 
           scale_factor=SCALE_FACTOR, 
           material_type='terrain',
           bbox=(-63.679503,44.564071,-63.565863,44.625435)
           )

Overall DEM processing:   0%|          | 0/2689 [00:00<?, ?rows/s]

Generating vertices:   0%|          | 0/1345 [00:00<?, ?row/s]

Generating mesh topology:   0%|          | 0/1344 [00:00<?, ?row/s]

In [7]:
# 3. Add the DEM layer to the stage
add_layer_to_stage(stage, dem_layer)


Mesh Statistics:
Number of points: 3351740
Number of faces: 6695808
Z range: -0.9736483693122864 to 133.67996215820312


# 3DTiles Buildings in USD

In [12]:
blg_path = Path("C:/Users/ymoisan/3dtiles/Cesium-1.123/Halifax_0_0_1_aoi_3D_4326/tileset.json")

In [ ]:
bldg_layer = create_layer_from_3dtiles(blg_path, scale_factor=1.0, z_offset=50.0)

In [ ]:
add_layer_to_stage(
    stage,
    bldg_layer,
    "/World/Buildings",  # Different path in the USD hierarchy
)

# GPKG Buildings in USD

In [8]:
# Create and add buildings layer from GeoPackage
buildings_gpkg_layer = create_layer_from_geopackage("halifax-aoi-plus_4326.gpkg", material_type='building', scale_factor=SCALE_FACTOR, z_offset=50.0)

In [ ]:
# Create and add buildings layer from GeoPackage
add_layer_to_stage(stage, buildings_gpkg_layer, "/World/Buildings")

# Save stage

In [8]:
# 6. Save the stage once after all layers are added
save_stage(stage)

Writing USD file... |

# Odd stuff

In [3]:
def convert_usdc_to_usdz(usdc_path: str, usdz_path: str):
    # Open the existing USDC stage
    stage = Usd.Stage.Open(usdc_path)
    if not stage:
        raise RuntimeError(f"Could not open stage at {usdc_path}")
    
    # Package the stage into a USDZ file.
    # (This requires a USD build with USDZ support)
    UsdUtils.CreateNewUsdzPackage(stage, usdz_path)
    print(f"Converted {usdc_path} to {usdz_path}")